In [1]:
from neucube import Reservoir
from neucube.encoder import Delta
from neucube.validation import Pipeline
from neucube.sampler import SpikeCount
from neucube.datamanager import DataManager
import torch

In [2]:
params = {
            'source_data_path': 'example_data/labquake_source',
            'samples_path':'example_data/labquake_samples',
            'sampling_rate': 5000,
            'batch_duration': 12000,
    
        }
datamanager = DataManager(params)

In [3]:
datamanager.process_and_save_batches()

KeyboardInterrupt: 